# Sentiment Analysis Mini-Challenge

## 1. Introduction
Sentiment analysis is a crucial task in Natural Language Processing (NLP) that involves determining the sentiment or tone of a given text. It has numerous applications, such as understanding customer feedback, monitoring social media sentiment, and analyzing product reviews. However, manually labeling large datasets for sentiment analysis can be time-consuming and costly. Semi-supervised learning techniques, such as weak supervision, can help alleviate this challenge by leveraging a small amount of labeled data along with a larger set of unlabeled data to improve model performance.

 ## 2. Dataset Selection and Exploratory Data Analysis
The dataset used for this mini-challenge is the Amazon Polarity dataset, which consists of product reviews from Amazon labeled as either positive or negative. The dataset is loaded using the Hugging Face Datasets library. Exploratory data analysis is performed to gain insights into the distribution of labels, length of reviews, and other relevant characteristics.

As the dataset contains 4 million reviews we cut this down into a subset of 6666 reviews for the purpose of this mini-challenge. 666 of the reviews are used for validation, the remaining 6000 are split into 1000 labeled samples and 5000 artificially unlabeled samples.

Each subset has a 50/50 split of positive and negative reviews. 

In [ ]:
import seaborn as sns

from src.data_loader import load_datasets

train_df, unlabeled, validation = load_datasets("../data/partitions")

In [ ]:
print(f"Labeled Dataset Length: {len(train_df)}")
print(f"Unlabeled Dataset Length: {len(unlabeled)}")
print(f"Validation Dataset Length: {len(validation)}")

To get a better idea of the whole dataset, we will merge the data again and perform some exploratory data analysis.

In [ ]:
import pandas as pd
from src.data_loader import LABEL_MAP

# for eda purposes

unlabeled.rename(columns={'ground_truth': 'label'}, inplace=True)


eda_df = pd.concat([train_df, unlabeled, validation])

eda_df['label'] = eda_df['label'].map(LABEL_MAP)

In [ ]:
from matplotlib import pyplot as plt

eda_df['review_length'] = eda_df['content'].apply(len)

plt.figure(figsize=(10, 6))
sns.histplot(eda_df['review_length'], bins=50, kde=True)
plt.title('Distribution of Review Lengths in Training Data')
plt.xlabel('Review Length')
plt.ylabel('Frequency')
plt.show()



In [ ]:
eda_df

In [ ]:
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer


def plot_most_common_words(df, top_n=20):
    df['label'] = df['label'].map({v: k for k, v in LABEL_MAP.items()})
    pos_reviews = df[df['label'] == 1]['content']
    neg_reviews = df[df['label'] == 0]['content']

    vectorizer_pos = CountVectorizer(stop_words='english')
    vectorizer_neg = CountVectorizer(stop_words='english')

    pos_word_count = vectorizer_pos.fit_transform(pos_reviews)
    neg_word_count = vectorizer_neg.fit_transform(neg_reviews)

    pos_sum_words = pos_word_count.sum(axis=0)
    neg_sum_words = neg_word_count.sum(axis=0)

    pos_words_freq = [(word, pos_sum_words[0, idx]) for word, idx in
                      zip(vectorizer_pos.get_feature_names_out(), range(pos_sum_words.shape[1]))]
    neg_words_freq = [(word, neg_sum_words[0, idx]) for word, idx in
                      zip(vectorizer_neg.get_feature_names_out(), range(neg_sum_words.shape[1]))]

    pos_words_freq = sorted(pos_words_freq, key=lambda x: x[1], reverse=True)
    neg_words_freq = sorted(neg_words_freq, key=lambda x: x[1], reverse=True)

    words, freq = zip(*pos_words_freq[:top_n])
    plt.figure(figsize=(10, 5))
    plt.bar(words, freq)
    plt.title('Most common words in positive reviews')
    plt.xticks(rotation=90)
    plt.show()

    words, freq = zip(*neg_words_freq[:top_n])
    plt.figure(figsize=(10, 5))
    plt.bar(words, freq)
    plt.title('Most common words in negative reviews')
    plt.xticks(rotation=90)
    plt.show()

plot_most_common_words(eda_df, top_n=20)


In [ ]:
def generate_word_cloud(text, title):
    wordcloud = WordCloud(width=800, height=800,
                          background_color='white',
                          stopwords=set(STOPWORDS),
                          min_font_size=10).generate(text)

    plt.figure(figsize=(8, 8), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.title(title)
    plt.show()



In [ ]:
pos_reviews_text = " ".join(eda_df[eda_df['label'] == "positive"]['content'])
generate_word_cloud(pos_reviews_text, "Word Cloud for Positive Reviews")

In [ ]:
neg_reviews_text = " ".join(eda_df[eda_df['label'] == "negative"]['content'])
generate_word_cloud(neg_reviews_text, "Word Cloud for Negative Reviews")


## 3. Data Splitting Strategy
The dataset is split into development, validation, labeled, and unlabeled sets using a nested split approach. The development set is a fraction of the full dataset, the validation set is a fraction of the test dataset, and the labeled set is a fraction of the development set. The remaining samples in the development set are considered unlabeled. The nested split always adds in 25% increments (25, 50, 75), and a 1/6 split is used, resulting in 1000 labeled and 5000 weakly labeled samples in total.

Given the focus of the MC on the impact of weak labelling and its impact, we introduce a nested split which further divides our training data into splits.

The goal is to identify the optimal amount of additional data that can be used to improve model performance without the need for manual annotation.

## 4. Baseline Model Performance
A pretrained language model, specifically `sentence-transformers/all-MiniLM-L6-v2`, is used as the baseline model for sentiment classification without training. The baseline model's performance is evaluated and the implications of the results are discussed.

## 5. Supervised Learning Performance
Supervised learning techniques, including transfer learning and fine-tuning, are applied to the sentiment classification task using different amounts of labeled samples. The impact of the number of labeled samples on model performance is analyzed and presented.

### 5.1 Transfer Learning
The performance of transfer learning using different amounts of labeled samples is presented and discussed.

### 5.2 Fine-tuning
The performance of fine-tuning using different amounts of labeled samples is presented and discussed.

## 6. Semi-Supervised Learning Performance
Semi-supervised learning techniques, specifically K-Nearest Neighbors (KNN) and Logistic Regression (LogReg), are employed to generate weak labels for the unlabeled samples. The impact of the number of labeled samples and weak labeling strategies on model performance is analyzed and presented.

### 6.1 K-Nearest Neighbors (KNN)
The performance of KNN-based weak labeling using different amounts of labeled samples is presented and discussed.

### 6.2 Logistic Regression (LogReg)
The performance of LogReg-based weak labeling using different amounts of labeled samples is presented and discussed.

## 7. Learning Curve Analysis
The learning curve, plotting the model performance against varying numbers of labeled samples for each technique (supervised and semi-supervised), is presented and analyzed. The focus is on the range with few labeled samples, and the practical implications of the results are discussed.

In [ ]:
# Code for generating the learning curve plot

## 8. Model Comparison and Analysis
A thorough analysis of the results is conducted, comparing the baseline model, supervised learning techniques, and semi-supervised learning techniques. The impact of different weak labeling strategies and training data sizes on model performance is evaluated. The best approach for the chosen dataset is determined, emphasizing the models that achieve acceptable performance with few manually annotated samples.

## 9. Time Savings Factor and Implications
The time savings factor, quantifying the reduction in manually labeled data required to achieve acceptable performance levels using weak labeling approaches, is calculated. The implications of the findings are discussed.

## 10. Conclusion and Future Directions
The key findings, insights, and potential implications of the sentiment analysis mini-challenge are summarized. The effectiveness of weak supervision techniques in reducing the need for manual annotation while maintaining acceptable model performance is discussed. Future directions for research and improvements are outlined.

## 11. AI Tool Usage Assessment
The use of ChatGPT or other AI tools throughout the mini-challenge is documented and assessed. The tasks for which they were used, the prompting strategies employed, and their contribution to solving the problem and acquiring new skills are specified.